In [16]:
import pandas as pd

In [17]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
test_ids = test['PassengerId']

In [18]:
print(train.shape)
print(test.shape)

(8693, 14)
(4277, 13)


In [19]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [20]:
train.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported'],
      dtype='object')

In [21]:
def clean(df: pd.DataFrame):
    df = df.drop(['PassengerId', 'Cabin', 'Name'], axis=1)
    cats = ['HomePlanet', 'Destination']
    to_num = ['CryoSleep', 'VIP']
    for cat in cats:
        df[cat] = le.fit_transform(df[cat])
        print(le.classes_)
    for col in to_num:
        df[col] = df[col].astype(bool).astype(int)
    df = df.fillna(df.median())
    return df

In [22]:
train = clean(train)
test = clean(test)

['Earth' 'Europa' 'Mars' nan]
['55 Cancri e' 'PSO J318.5-22' 'TRAPPIST-1e' nan]
['Earth' 'Europa' 'Mars' nan]
['55 Cancri e' 'PSO J318.5-22' 'TRAPPIST-1e' nan]


In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [24]:
X = train.drop('Transported', axis=1)
y = train['Transported']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
clf = LogisticRegression(random_state=42, max_iter=1000).fit(X_train, y_train)
predictions = clf.predict(X_val)
from sklearn.metrics import accuracy_score
accuracy_score(y_val, predictions)

0.772857964347326

In [26]:
submission = clf.predict(test)
df = pd.DataFrame({'PassengerId': test_ids.values,
                    'Transported': submission,})
df['Transported'] = df['Transported'].astype(bool)
df.head(5)

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,False


In [28]:
df.to_csv('PabloSegovia_SpaceshipTitanic.csv', index=False)